# Estimate a starting ρ from the length model by maximum likelihood.

Independent lossmodels
Length model The independent loss length model is a
Markov process known as an M/M/∞ queuing model
[28] (Figure 2A). In this queuing model, customers
(i.e., spacers) arrive according to a Poisson process with
rate λ. They are immediately served and exit after an
exponential waiting time with rate μ. The stationary distribution
of the number of busy servers (i.e., the number
of spacers in the array), is a Poisson distribution with
rate ρ:

ro = $$\rho = \frac{\lambda}{\mu}$$
with $\lambda$ = spacer insertion rate, $\mu$ = spacer_deletion_rate

prob_n_given_ro = $$p(n|\rho) = e^{-\rho} \frac{\rho^n}{n!}$$

In [3]:
from CRISPR_functions import rho, prob_n_given_ro
n=5
lbd=2
mu=4

prob_n_given_ro(n,rho(lbd,mu))

0.0001579506926334983